In [ ]:
#If you want to change the dataset, go to:
#dataset directory -> trainA and delete its content
#dataset directory -> trainB and delete its content
#dataset directory -> testA and delete its content
#dataset directory -> testB and delete its content
#and then do this:

In [ ]:
#We took the CycleGAN dataset, you can take other datasets from the CycleGAN model
#Mount your google drive and then add this cell:
%cd CycleGAN
!bash ./datasets/download_cyclegan_dataset.sh horse2zebra

In [ ]:
#We modified the MRI dataset and saved 80% of it in training directory. For another dataset change the path of the 'directory'.
#After all the instalations add the next 3 cells:
import tensorflow as tf
import os

# Define directory
directory = '/content/drive/MyDrive/CycleGANTrained/MRIDataSet/Training/pituitary_tumor'

# List all the file paths
file_paths = [os.path.join(directory, img) for img in os.listdir(directory)]

# Create a dataset from these paths
dataset = tf.data.Dataset.from_tensor_slices(file_paths)

# Define a function to load, decode, resize, and normalize the image
def process_img(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=1)  # change this if your images are not in jpeg format
    img = tf.image.resize(img, [256, 256])  # resizing the image
    img = tf.cast(img, tf.float32)
    img /= 255.0  # normalize to [0,1] range
    return img
# Apply this function to all items in the dataset
dataset = dataset.map(process_img)
output_dir = '/content/drive/MyDrive/CycleGANTrained/dataset/trainA'
os.makedirs(output_dir, exist_ok=True)  # create the output directory if it doesn't exist

for i, image in enumerate(dataset):
    output_path = os.path.join(output_dir, f'image_{i}.jpg')
    image = tf.cast(image * 255.0, tf.uint8)  # scale back to [0, 255] range
    encoded_image = tf.image.encode_jpeg(image)
    tf.io.write_file(output_path, encoded_image)


In [ ]:
#We modified the MRI dataset and saved 20% of it in testing directory.For another dataset change the path of the 'directory'.
import tensorflow as tf
import os

# Define directory
directory = '/content/drive/MyDrive/CycleGANTrained/MRIDataSet/Testing/pituitary_tumor'

# List all the file paths
file_paths = [os.path.join(directory, img) for img in os.listdir(directory)]

# Create a dataset from these paths
dataset = tf.data.Dataset.from_tensor_slices(file_paths)

# Define a function to load, decode, resize, and normalize the image
def process_img(file_path):
    # Load the raw data from the file
    img = tf.io.read_file(file_path)
    # Decode the image file to a tensor and set color channels to 1 (grayscale)
    img = tf.image.decode_jpeg(img, channels=1)
    # Resize the image to 256 x 256 to fit the CycleGAN model requirements
    img = tf.image.resize(img, [256, 256])
    # Convert the pixel values to float32 type
    img = tf.cast(img, tf.float32)
    # Normalize the pixel values to the range [0, 1]
    img /= 255.0
    return img

# Apply the process_img function to all images in the dataset
dataset = dataset.map(process_img)

# Define the output directory where the processed images will be stored
output_dir = '/content/drive/MyDrive/CycleGANTrained/dataset/testA'
os.makedirs(output_dir, exist_ok=True)

for i, image in enumerate(dataset):
    # Define the output path for each image
    output_path = os.path.join(output_dir, f'image_{i}.jpg')
    # Scale the pixel values back to the original range [0, 255]
    image = tf.cast(image * 255.0, tf.uint8)
    # Encode the image to JPEG format
    encoded_image = tf.image.encode_jpeg(image)
    # Save the processed image to the output path
    tf.io.write_file(output_path, encoded_image)

In [ ]:
#We modified the zebra dataset
import os
import cv2

# Define directory for the horse dataset
horse_directory = '/content/drive/MyDrive/CycleGANTrained/dataset/trainB'

# List all the horse file paths
horse_file_paths = [os.path.join(horse_directory, img) for img in os.listdir(horse_directory)]

# Convert horse images to grayscale
for file_path in horse_file_paths:
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    cv2.imwrite(file_path, img)

# Define directory for the zebra dataset
zebra_directory = '/content/drive/MyDrive/CycleGANTrained/dataset/testB'

# List all the zebra file paths
zebra_file_paths = [os.path.join(zebra_directory, img) for img in os.listdir(zebra_directory)]

# Convert zebra images to grayscale
for file_path in zebra_file_paths:
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    cv2.imwrite(file_path,img)

In [ ]:
#Now you can train on other datasets.

In [ ]:
#For the regular CycleGAN training (200 epochs): (change the epoch count according to the last saved chakpoint)
!python train.py --dataroot /content/drive/MyDrive/CycleGANTrained/dataset --name MRItoZebra --model cycle_gan --continue_train --epoch_count 1

In [ ]:
# If you want to keep training with constant learning rate add: (change the epoch count according to the last saved chakpoint)
!python train.py --dataroot /content/drive/MyDrive/CycleGANTrained/dataset --name MRItoZebra --model cycle_gan --continue_train --epoch_count 265 --n_epochs 300 --n_epochs_decay 200 --lr 0.0002
#In this command, We set n_epochs_decay to 10000.
#This means that the learning rate will start decaying only after 10000 epochs, which effectively makes the learning rate constant at 0.0002 for the training duration of less than 1000 epochs.
#If you want more than 10000 just change the n_epochs_decay number.

In [ ]:
#If you want to keep training with learning rate decay phase: (change the epoch count according to the last saved chakpoint)
!python train.py --dataroot /content/drive/MyDrive/CycleGANTrained/dataset --name MRItoZebra --model cycle_gan --continue_train --epoch_count 346 --n_epochs 295 --n_epochs_decay 200 --lr 0.0002 --display_id 0
#epoch_count is on which epoch we are now, n_epochs is the number of epochs with a constant learning rate and n_epochs_decay is the count of epochs from now until the learning rate will get to 0.
